In [22]:
!pip install pytesseract

In [3]:
import cv2
import pandas as pd
from ultralytics import YOLO
import cvzone
import numpy as np
import pytesseract
from datetime import datetime

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

license_model = YOLO('license_best.pt')

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        point = [x, y]
        print(point)

#cv2.namedWindow('RGB')

# Calculate the screen width and height
screen_width, screen_height = 1920, 900  # Set your screen resolution

# Resize the window to fit the screen
cv2.namedWindow('RGB', cv2.WINDOW_NORMAL)
cv2.resizeWindow('RGB', screen_width, screen_height)


cv2.setMouseCallback('RGB', RGB)

#cap = cv2.VideoCapture(r"C:\Users\aliye\OneDrive\Desktop\tests\VIDEOS\close.MOV")
#cap = cv2.VideoCapture(r"C:\Users\aliye\Downloads\car-result-with-counter (1).mp4")
cap = cv2.VideoCapture(r"C:\Users\aliye\OneDrive\Desktop\tests\wrong.mp4")
my_file = open(r"C:\Users\aliye\OneDrive\Desktop\tests\plates\classes.txt", "r")
data = my_file.read()
class_list = data.split("\n") 

# Reverse operation for area
area = [(1320, 830), (895, 830), (895, 855), (1320, 855)]

area2 = [(480, 789), (880, 789), (880, 814), (480, 814)]

count = 0
list1 = []
processed_numbers = set()

# Open file for writing car plate data
car_plate_data = []

object_inside_area = False
prev_cx, prev_cy = 0, 0
direction = None  # Variable to store the direction of object movement

while True:
    ret, frame = cap.read()
    count += 1
    if count % 3 != 0:
        continue
    if not ret:
        break
    
    frame = cv2.resize(frame, (1920, 1080))
    results = license_model.predict(frame)
    a = results[0].boxes.data
    a = a.cpu()
    px = pd.DataFrame(a).astype("float")
    
    for index, row in px.iterrows():
        x1, y1, x2, y2, _, d = map(int, row)
        c = class_list[d]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        
        result = cv2.pointPolygonTest(np.array(area, np.int32), (cx, cy), False)
        result2 = cv2.pointPolygonTest(np.array(area2, np.int32), (cx, cy), False)  # New polygon area

        if result >= 0:  # Object is inside the predefined area
            if not object_inside_area and prev_cx != 0 and prev_cy != 0:
                if prev_cy < cy:
                    direction = "top_to_bottom"
                elif prev_cy > cy:
                    direction = "bottom_to_top"
                
                if direction == "top_to_bottom":
                    crop = frame[y1:y2, x1:x2]
                    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
                    gray = cv2.bilateralFilter(gray, 10, 20, 20)

                    text = pytesseract.image_to_string(gray).strip()
                    text = text.replace('(', '').replace(')', '').replace(',', '').replace('|','').replace('_','').replace('}','').replace('—','')
                    if text not in processed_numbers:
                        processed_numbers.add(text) 
                        list1.append(text)
                        current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        date, time = current_datetime.split(' ')
                        car_plate_data.append([text, date, time])
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
                        cv2.imshow('crop', crop)

            object_inside_area = True
        elif result2 >= 0:  # Object is inside the new predefined area
            if not object_inside_area and prev_cx != 0 and prev_cy != 0:
                if prev_cy < cy:
                    direction = "top_to_bottom"
                elif prev_cy > cy:
                    direction = "bottom_to_top"
                
                if direction == "bottom_to_top":
                    crop = frame[y1:y2, x1:x2]
                    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
                    gray = cv2.bilateralFilter(gray, 10, 20, 20)

                    text = pytesseract.image_to_string(gray).strip()
                    text = text.replace('(', '').replace(')', '').replace(',', '').replace('|','').replace('_','').replace('}','').replace('—','')
                    if text not in processed_numbers:
                        processed_numbers.add(text) 
                        list1.append(text)
                        current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        date, time = current_datetime.split(' ')
                        car_plate_data.append([text, date, time])
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
                        cv2.imshow('crop', crop)

            object_inside_area = True
        else:  # Object is outside the predefined area
            object_inside_area = False
        
        prev_cx, prev_cy = cx, cy
    
    cv2.polylines(frame, [np.array(area, np.int32)], True, (0, 0, 255), 2)
    cv2.polylines(frame, [np.array(area2, np.int32)], True, (255, 0, 0), 2)  # New polygon area
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()   
cv2.destroyAllWindows()

# Create a DataFrame from the collected data
car_plate_df = pd.DataFrame(car_plate_data, columns=['NumberPlate', 'Date', 'Time']).assign(**{'Violation type': 'driving in opposite direction'})

# Save the DataFrame to an Excel file
car_plate_df.to_excel('C:\\Users\\aliye\\OneDrive\\Desktop\\Violations\\car_plate_data.xlsx', index=False)



# the following is set based on your video background

#best
#top_bottom_area = [(895, 830), (1320, 830), (1320, 855), (895, 855)]

#best
#bottom_top_area = [(480, 830), (880, 830), (880, 855), (480, 855)]



0: 384x640 (no detections), 97.5ms
Speed: 6.0ms preprocess, 97.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
[946, 790]

0: 384x640 (no detections), 91.0ms
Speed: 0.0ms preprocess, 91.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 85.3ms
Speed: 4.4ms preprocess, 85.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
[1278, 1054]
[1270, 1046]

0: 384x640 (no detections), 79.1ms
Speed: 5.7ms preprocess, 79.1ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)
[1265, 1049]
[1265, 1051]

0: 384x640 (no detections), 85.5ms
Speed: 0.0ms preprocess, 85.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 84.7ms
Speed: 0.0ms preprocess, 84.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 86.7ms
Speed: 0.0ms preprocess, 86.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 38